## Get started with SageMaker

This notebok is part of [Introductory blog on SageMaker Core](https://aws.amazon.com/blogs/machine-learning/introducing-sagemaker-core-a-new-object-oriented-python-sdk-for-amazon-sagemaker/).

In this notebook you'll learn how SageMaker can be used to:

1. Preprocess (and optionally explore) a dataset
2. Train an XGBoost classifier for customer churn prediction, using a managed job with SageMaker Training, using a managed image.
3. Create a managed real-time SageMaker endpoint.

All SageMaker resources are created using the SageMaker Core SDK. You can find more information about sagemaker-core [here](https://sagemaker-core.readthedocs.io/en/latest/)

In [33]:
%pip install --upgrade pip -q
%pip install sagemaker-core -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
import time
import io
from datetime import datetime
import pandas as pd
from sagemaker_core.helper.session_helper import Session, get_execution_role
from io import StringIO
import pandas as pd
from sagemaker_core.main.shapes import (
    ProcessingInput,
    ProcessingResources,
    AppSpecification,
    ProcessingS3Input,
    ProcessingOutputConfig
)
from sagemaker_core.shapes import (
    ProcessingResources,
    ProcessingClusterConfig,
    ProcessingOutput,
    ProcessingS3Output,
)

from sagemaker_core.resources import ProcessingJob

In [35]:
# Set up region, role and bucket parameters used throughout the notebook.
sagemaker_session = Session()
region = sagemaker_session.boto_region_name
role = get_execution_role()
bucket = sagemaker_session.default_bucket()
s3_client = sagemaker_session.boto_session.client('s3')

print(f"AWS region: {region}")
print(f"Execution role: {role}")
print(f"Default S3 bucket: {bucket}")

AWS region: us-east-1
Execution role: arn:aws:iam::590184044598:role/service-role/SageMaker-ExecutionRole-20250115T132367
Default S3 bucket: sagemaker-us-east-1-590184044598


## Preprocess dataset
We'll use a synthetic dataset that AWS provides for customer churn prediction.


<div class="alert alert-block alert-info">
<b>NOTE:</b> This sample doesn't perform any exploratory data anlysis since how to preprocess the dataset is already known.
    
If you're interested in how to perform exploratory analysis, there's a section in the documentation for the sagemaker-python-sdk available that explores the dataset, [here](https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.html).
</div>

## Upload the processing code to S3

The pre-processing code is already available in the current directory with the name `preprocess.py`. Have a look at the code to understand the processing logic.

In [36]:
pre_processing_code_s3_uri = sagemaker_session.upload_data("preprocess.py", key_prefix="sagemaker-core-intro-blog/processing/code")
pre_processing_code_s3_uri

's3://sagemaker-us-east-1-590184044598/sagemaker-core-intro-blog/processing/code/preprocess.py'

### Create S3 variables for holding the processed data

Create S3 variables for holding the processed data (train, validation and test).

In [37]:
processed_train_data_uri = f"s3://{bucket}/sagemaker-core-intro-blog/processing/output/train"
processed_validation_data_uri = f"s3://{bucket}/sagemaker-core-intro-blog/processing/output/validation"
processed_test_data_uri = f"s3://{bucket}/sagemaker-core-intro-blog/processing/output/test"

## Create processing job

Below code submits a sagemaker processing job.

1. ProcessingResources provides processing cluster details.
2. AppSpecification provides processing container details. Here SKlearn processing container provided by sagemaker is used.
3. Two objects of ProcessingInput specifying code and input data locations and configurations.
4. ProcessingOutputConfig provides details on where the processed data will be stored.

In [38]:
# Initialize a ProcessingJob resource
current_timestamp = datetime.now()
formatted_timestamp = current_timestamp.strftime("%Y-%m-%d-%H-%M-%S")

processing_job = ProcessingJob.create(
    processing_job_name=f"sagemaker-core-data-prep-{formatted_timestamp}",
    processing_resources=ProcessingResources(
        cluster_config=ProcessingClusterConfig(
            instance_count=1,
            instance_type="ml.m5.xlarge",
            volume_size_in_gb=20
        )
    ),
    app_specification=AppSpecification(
        image_uri=f"683313688378.dkr.ecr.{region}.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3",
        container_entrypoint=["python3", "/opt/ml/processing/code/preprocess.py"]
    ),
    role_arn=role,  # Intelligent default for execution role
    processing_inputs=[
        ProcessingInput(
            input_name="input",
            s3_input=ProcessingS3Input(
                s3_uri=f"s3://sagemaker-example-files-prod-{region}/datasets/tabular/synthetic/churn.txt",
                s3_data_type="S3Prefix",
                local_path="/opt/ml/processing/input",
                s3_input_mode="File"
            ),
        ),
        ProcessingInput(
            input_name="code",
            s3_input=ProcessingS3Input(
                s3_uri=pre_processing_code_s3_uri,
                s3_data_type="S3Prefix",
                local_path="/opt/ml/processing/code",
                s3_input_mode="File"
            ),
        )
    ],
    processing_output_config= ProcessingOutputConfig(
            outputs=[
                ProcessingOutput(
                    output_name="train",
                    s3_output=ProcessingS3Output(
                        s3_uri=processed_train_data_uri,
                        s3_upload_mode="EndOfJob",
                        local_path="/opt/ml/processing/output/train"
                    )
                ),
                ProcessingOutput(
                    output_name="validation",
                    s3_output=ProcessingS3Output(
                        s3_uri=processed_validation_data_uri,
                        s3_upload_mode="EndOfJob",
                        local_path="/opt/ml/processing/output/validation"
                    )
                ),
                ProcessingOutput(
                    output_name="test",
                    s3_output=ProcessingS3Output(
                        s3_uri=processed_test_data_uri,
                        s3_upload_mode="EndOfJob",
                        local_path="/opt/ml/processing/output/test"
                    )
                )
            ]
        )
)

# Wait for the ProcessingJob to complete
processing_job.wait()


[03/31/25 06:49:29] INFO     Creating processing_job resource.                                   ]8;id=25841;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=508799;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#25812\25812]8;;\

Output()

[03/31/25 06:52:01] INFO     Final Resource Status: Completed                                    ]8;id=355484;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=810631;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#26024\26024]8;;\

## Train a classifier using XGBoost
Use SageMaker Training and the managed XGBoost image to train a classifier. <br />
More details on how to use SageMaker managed training with XGBoost can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html).

<div class="alert alert-block alert-info">
  <b>NOTE:</b> For more information on using SageMaker managed container images and retrieving their ECR paths, 
  <a href="https://docs.aws.amazon.com/sagemaker/latest/dg-ecr-paths/sagemaker-algo-docker-registry-paths.html" target="_blank">here</a> 
  is the documentation. Please note that the image URI might need to be updated based on your selected AWS region.
</div>


In [39]:
image = f"683313688378.dkr.ecr.{region}.amazonaws.com/sagemaker-xgboost:1.7-1"

In [40]:
from sagemaker_core.resources import TrainingJob
from sagemaker_core.shapes import (
    AlgorithmSpecification,
    Channel,
    DataSource,
    S3DataSource,
    ResourceConfig,
    StoppingCondition,
    OutputDataConfig,
)

job_name = "xgboost-churn-" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)  # Name of training job
instance_type = "ml.m4.xlarge"  # SageMaker instance type to use for training
instance_count = 1  # Number of instances to use for training
volume_size_in_gb = 30  # Amount of storage to allocate to training job
max_runtime_in_seconds = 600  # Maximum runtimt. Job exits if it doesn't finish before this
s3_output_path = f"s3://{bucket}"  # bucket and optional prefix where the training job stores output artifacts, like model artifact.

# Specify hyperparameters
hyper_parameters = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.8",
    "verbosity": "0",
    "objective": "binary:logistic",
    "num_round": "100",
}

# Create training job.
training_job = TrainingJob.create(
    training_job_name=job_name,
    hyper_parameters=hyper_parameters,
    algorithm_specification=AlgorithmSpecification(
        training_image=image, training_input_mode="File"
    ),
    role_arn=role,
    input_data_config=[
        Channel(
            channel_name="train",
            content_type="csv",
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type="S3Prefix",
                    s3_uri=processed_train_data_uri,
                    s3_data_distribution_type="FullyReplicated",
                )
            ),
        ),
        Channel(
            channel_name="validation",
            content_type="csv",
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type="S3Prefix",
                    s3_uri=processed_validation_data_uri,
                    s3_data_distribution_type="FullyReplicated",
                )
            ),
        ),
    ],
    output_data_config=OutputDataConfig(s3_output_path=s3_output_path),
    resource_config=ResourceConfig(
        instance_type=instance_type,
        instance_count=instance_count,
        volume_size_in_gb=volume_size_in_gb,
    ),
    stopping_condition=StoppingCondition(max_runtime_in_seconds=max_runtime_in_seconds),
)

# Wait for the training job to complete
# training_job.wait()
training_job.wait(poll=60, timeout=None, logs=False)

[03/31/25 06:52:08] INFO     Creating training_job resource.                                     ]8;id=644411;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=271728;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#28129\28129]8;;\

Output()

[03/31/25 06:55:08] INFO     Final Resource Status: Completed                                    ]8;id=367444;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=45178;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#28415\28415]8;;\

## Use model artifacts for real time inference
To use the model to perform real time inference, we need to:

1. Create a SageMaker model with the same first-party image as we used for training, and the model artifacts produced during training. Indeed, such image can also be used to run inference
2. Create an `EndpointConfig` using the SageMaker model object created in the previous step. The endpoint configuration specifies what SageMaker model to use, and what endpoint type.
3. Create a SageMaker endpoint using `EndpointConfig` and other optional parameters.

More information about SageMaker Endpoints can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html).

#### Create SageMaker Model

Create a Model resource based on the model artifacts produced by the training job.

In [45]:
from sagemaker_core.resources import Model
from sagemaker_core.shapes import ContainerDefinition

model_s3_uri = training_job.model_artifacts.s3_model_artifacts  # Get URI of model artifacts from the training job.

model_name = f"customer-churn-xgboost-{time.strftime('%Y-%m-%d-%H-%M-%S', time.gmtime())}"

# Create SageMaker model with the unique name
customer_churn_model = Model.create(
    model_name=model_name,  # Use the variable here, not the hardcoded string
    primary_container=ContainerDefinition(image=image, model_data_url=model_s3_uri),
    execution_role_arn=role,
)

print(f"Successfully created model: {model_name}")

[03/31/25 07:01:00] INFO     Creating model resource.                                            ]8;id=415815;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=935532;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#17871\17871]8;;\

Successfully created model: customer-churn-xgboost-2025-03-31-07-01-00


## Create endpoint configuration for real-time inference
To create a SageMaker endpoint we first create an `EndpointConfig`. The endpoint configuration specifies what SageMaker model to use.

In [52]:
from sagemaker_core.resources import Endpoint, EndpointConfig
from sagemaker_core.shapes import ProductionVariant
import time
import boto3

# Create timestamp for unique naming
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# Use timestamp in endpoint config name
endpoint_config_name = f"churn-prediction-endpoint-config-{timestamp}"  
model_name = customer_churn_model.get_name()  # Get name of SageMaker model created in previous step
endpoint_name = f"customer-churn-endpoint-{timestamp}"  # Use timestamp in endpoint name

# Create the endpoint configuration
endpoint_config = EndpointConfig.create(
    endpoint_config_name=endpoint_config_name,
    production_variants=[
        ProductionVariant(
            variant_name="AllTraffic",
            model_name=model_name,
            instance_type=instance_type,
            initial_instance_count=1,
        )
    ],
)
print(f"Endpoint config created: {endpoint_config_name}")

# Create the endpoint using the endpoint config
endpoint = Endpoint.create(
    endpoint_name=endpoint_name,
    endpoint_config_name=endpoint_config_name,
)
print(f"Endpoint creation initiated: {endpoint_name}")

# Wait for the endpoint to become available
sagemaker_client = boto3.client('sagemaker')
print(f"Waiting for endpoint {endpoint_name} to become InService...")

# Poll until the endpoint becomes available or fails
status = "Creating"
while status == "Creating":
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = response['EndpointStatus']
    
    if status == "Creating":
        print(f"Endpoint status: {status} - waiting 30 seconds...")
        time.sleep(30)
    elif status == "InService":
        print(f"Endpoint is now available! Status: {status}")
    else:
        print(f"Endpoint creation failed or entered unexpected state: {status}")
        print(f"Failure reason: {response.get('FailureReason', 'No failure reason provided')}")
        break

[03/31/25 07:11:53] INFO     Creating endpoint_config resource.                                   ]8;id=534248;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=720063;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#9735\9735]8;;\

Endpoint config created: churn-prediction-endpoint-config-2025-03-31-07-11-53


                    INFO     Creating endpoint resource.                                          ]8;id=920108;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=7326;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#8916\8916]8;;\

Endpoint creation initiated: customer-churn-endpoint-2025-03-31-07-11-53
Waiting for endpoint customer-churn-endpoint-2025-03-31-07-11-53 to become InService...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint status: Creating - waiting 30 seconds...
Endpoint is now available! Status: InService


In [54]:
# sagemaker_endpoint.wait_for_status(
#     target_status="InService"
# )  # Wait for endpoint to become in service

#### Test live endpoint - with a sample record from test dataset

Let us download the test data from S3.

In [56]:
# List objects in your bucket to find the correct path
response = s3.list_objects_v2(
    Bucket=bucket,
    Prefix="sagemaker-core-intro-blog/processing/output/"
)

# Print available paths
for obj in response.get('Contents', []):
    print(obj['Key'])

sagemaker-core-intro-blog/processing/output/test/test.csv
sagemaker-core-intro-blog/processing/output/train/train.csv
sagemaker-core-intro-blog/processing/output/validation/validation.csv


In [57]:
s3 = boto3.client("s3")
s3.download_file(
    Bucket=bucket,
    Key="sagemaker-core-intro-blog/processing/output/test/test.csv",  # Correct path
    Filename="test.csv"
)

#### Invoke the endpoint

Let us invoke the endpoint now.

In [62]:
import boto3
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def print_performance_metrics(predictions, target_column):
    """
    Print common performance metrics for binary classification.
    
    Parameters:
    predictions (DataFrame): DataFrame with model prediction probabilities
    target_column: The target column from the test dataset
    """
    # Convert probabilities to binary predictions (assuming 0.5 threshold)
    binary_predictions = (predictions > 0.5).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(target_column, binary_predictions)
    precision = precision_score(target_column, binary_predictions, zero_division=0)
    recall = recall_score(target_column, binary_predictions, zero_division=0)
    f1 = f1_score(target_column, binary_predictions, zero_division=0)
    
    try:
        auc = roc_auc_score(target_column, predictions)
    except:
        auc = "N/A"
    
    # Print results
    print(f"Performance Metrics:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"AUC:       {auc}")

# Load test data
df = pd.read_csv("test.csv", header=None)

# Assuming the last column is the target column (adjust if different)
test_target_column = df.iloc[:, -1]
print(f"Target column shape: {test_target_column.shape}")
print(f"Target column values: {test_target_column.value_counts()}")

# Pick a random record from CSV and convert it to string
sample = df.sample(1)
sample_payload = sample.to_csv(header=False, index=False).strip()

# Create a SageMaker runtime client for invoking the endpoint
runtime_client = boto3.client('sagemaker-runtime')

# Send sample payload to live endpoint and parse response
res = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,  # Use the endpoint name from earlier
    ContentType="text/csv",
    Body=sample_payload
)

result = res["Body"].read().decode("utf-8")
result = result.split("\n")[:-1]

# Compute performance metrics for this single prediction
df_result = pd.DataFrame(result).astype(float)

# Extract the target from the sample we used
sample_target = sample.iloc[:, -1]
print(f"Sample target value: {sample_target.values[0]}")
print(f"Model prediction: {df_result.values[0][0]}")

# For the single sample
print("\nSingle sample prediction:")
print(f"Actual value: {sample_target.values[0]}")
print(f"Predicted probability: {df_result.values[0][0]:.4f}")
print(f"Predicted class: {1 if df_result.values[0][0] > 0.5 else 0}")

Target column shape: (545,)
Target column values: 98
0.0    275
1.0    270
Name: count, dtype: int64
Sample target value: 1.0
Model prediction: 0.2609618306159973

Single sample prediction:
Actual value: 1.0
Predicted probability: 0.2610
Predicted class: 0


## Clean up

In [64]:
# Delete endpoint
endpoint.delete()

# Delete endpoint configuration
endpoint_config.delete()

# Delete model
customer_churn_model.delete()

print("Successfully deleted all SageMaker resources")

[03/31/25 07:56:58] INFO     Deleting Endpoint - customer-churn-endpoint-2025-03-31-07-11-53      ]8;id=31548;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=953556;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#9116\9116]8;;\

                    INFO     Deleting EndpointConfig -                                            ]8;id=173608;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=557145;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#9885\9885]8;;\
                             churn-prediction-endpoint-config-2025-03-31-07-11-53                                  

                    INFO     Deleting Model - customer-churn-xgboost-2025-03-31-07-01-00         ]8;id=181429;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=297606;file:///opt/conda/lib/python3.11/site-packages/sagemaker_core/main/resources.py#18018\18018]8;;\

Successfully deleted all SageMaker resources


In [65]:
# sagemaker_endpoint.delete()
# endpoint_config.delete()
# customer_churn_model.delete()